# Generate Meta Data and Preprocess Data from the MIMIC-III Demo Dataset for Conquery

This tutorial shows how data and meta data tables from the [MIMIC-III Demo Dataset](https://physionet.org/content/mimiciii-demo/1.4/) can be used to prepare data structures
needed for conquery.

In detail we will generate meta JSONs describing a table schema (Table-JSON), an import operation (Import-JSON, is much like the corresponding Table-JSON used for the preprocessing) and a concept (Concept-JSON, which offers the query functionality) from the table [DIAGNOSES_ICD.csv](https://physionet.org/files/mimiciii-demo/1.4/DIAGNOSES_ICD.csv).

Then we will use the Import-JSON to preprocess DIAGNOSES_ICD.csv to a DIAGNOSES_ICD.cqpp (**c**on**q**uery **p**re**p**rocessed).

Finally a dataset *MIMIC-III-Demo* will be created in an instance of conquery and the Table-JSON, Concept-JSON and DIAGNOSES_ICD.cqpp will be uploaded.

In [3]:
## The imports for this notebook
import pandas as pd
import io
import requests as r
import json
import numpy as np
import os
import re
from enum import Enum, auto
from jsonschema import Draft7Validator, RefResolver
from pathlib import Path
from zipfile import ZipFile
from io import BytesIO

# Define working directory
wd = Path(".")

## Helper Functions

In [ ]:
class CQTypes(Enum):
    STRING = auto()
    INTEGER = auto()
    BOOLEAN = auto()
    REAL= auto()
    DECIMAL= auto()
    MONEY= auto()
    DATE= auto()
    DATE_RANGE= auto()

def get_csv_name(url):
    filename_matcher = re.compile(r"[\w\d_-]+\.csv")
    match = filename_matcher.search(url)
    if not match:
        raise ValueError(f"Unable to extract file name from {url}")
    return match.group(0)


def typeConverter(dtype) :
    if np.issubdtype(dtype, np.object) :
        return CQTypes.STRING.name
    if np.issubdtype(dtype, np.integer) :
        return CQTypes.INTEGER.name
    if np.issubdtype(dtype, np.bool_) :
        return CQTypes.BOOLEAN
    if np.issubdtype(dtype, np.inexact) :
        return CQTypes.REAL
    # DECIMAL cannot be derived from the dtype because there is no analogon
    # MONEY cannot be derived from the dtype because it is a semantic rather than a logical type
    if np.issubdtype(dtype, np.datetime64):
        return CQTypes.DATE.name
    # DATE_RANGE not supported here yet
    raise ValueError(f"Encountered unhandled dtype: {dtype}")

def generate_table_column(name, dtype) :
    return {
        "name": name,
        "type" : typeConverter(dtype)
    }

def generate_table(name, df, primary_column) :
    return {
        "name" : name,
        "columns": [ generate_table_column(name, dtype) for name, dtype in zip(df.dtypes.keys().array, df.dtypes.values) if name != primary_column]
    }

def generate_import_column(name, dtype) :
    return {
        "inputColumn": name,
        "inputType": typeConverter(dtype),
        "name": name,
        "operation": "COPY"
    }

def generate_import(df, primary_column, source_file) :


    col_names = list(df.columns.values)
    col_names.remove(primary_column)
    non_primary_df = data_df[col_names]

    # Skip the filename suffix
    table_label = source_file.name.split(".")[0]

    return {
        "inputs": [
            {
                "output": [ generate_import_column(name, dtype) for name, dtype in zip(non_primary_df.dtypes.keys().array, non_primary_df.dtypes.values)],
                "primary": {
                    **generate_import_column(primary_column, df[[primary_column]].dtypes.values[0]),
                    "required": True,
                },
                "sourceFile": source_file.as_posix()
            }
        ],
        "table": table_label,
        "name": table_label
    }


"""
Create a validator from a base schema in the directory "./json_schema"
"""
def get_validator(base_schema_file):
    schema_store = {}

    directory = wd / "json_schema"
        
    for file in list(directory.glob("*.json")):
        
        with open(file, "r") as schema_file:
            schema = json.load(schema_file)
            schema_store[file.name] = schema

    resolver = RefResolver.from_schema(schema, store=schema_store)
    return Draft7Validator(schema_store[base_schema_file], resolver=resolver)

## Meta Data Creation
We will start with the creation of the meta data. For Table-JSON and Import-JSON we need the header of the data table (DIAGNOSES_ICD.csv), we want to use later in conquery.
This process is rather generic, as it is usually just an annotation of the columns with type information.

For the Concept-JSON we will use the meta data table (D_ICD_DIAGNOSES.csv) to create a tree structured concept from the hierachical *icd9_code*.

### Download Data Table

In [ ]:
data_url = "https://physionet.org/files/mimiciii-demo/1.4/DIAGNOSES_ICD.csv?download"
s=r.get(data_url).content
data_df = pd.read_csv(io.StringIO(s.decode('utf-8')), index_col="row_id", dtype={"subject_id": str, "hadm_id": str, "icd9_code": str })


# Write out the csv because it is needed for the preprocessing
data_file = wd / "data" / "csv" / get_csv_name(data_url)
data_file.parent.mkdir(parents=True, exist_ok=True)

data_df.to_csv(data_file)

### Generate Table-JSON and Validate

In [ ]:
table_name = data_file.name.split(".")[0]
table = generate_table(table_name, data_df, "subject_id")

get_validator("table.json").validate(table)

table_json_file = wd / "data" / "tables" / f"{table_name}.table.json"
table_json_file.parent.mkdir(parents=True, exist_ok=True)

with open(table_json_file, "w") as f:
    json.dump(table, f, indent="\t")

### Generate Import-JSON and Validate

In [ ]:
table_name = data_file.name.split(".")[0]
import_ = generate_import(data_df, "subject_id", data_file)

get_validator("import.json").validate(import_)

import_json_file = wd / "data" / "imports" / f"{table_name}.import.json"
import_json_file.parent.mkdir(parents=True, exist_ok=True)

with open(import_json_file, "w") as f:
    json.dump(import_, f, indent="\t")

### Generate Concept-JSON

In this section we generate an ICD concept based in the official ICD-9 catalog (https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Publications/ICD9-CM/2011/Dtab12.zip).
We could have used the meta data provided by MIMIC-III (https://physionet.org/files/mimiciii-demo/1.4/D_ICD_DIAGNOSES.csv), but it misses structural and descriptive information (such as chapters, names of higher hierarchies and additional infos).

In [ ]:
meta_url = "https://physionet.org/files/mimiciii-demo/1.4/D_ICD_DIAGNOSES.csv?download"
s=r.get(meta_url).content
meta_df = pd.read_csv(io.StringIO(s.decode('utf-8')), index_col="row_id", dtype={"subject_id": str, "icd9_code": str })


Download and extract the ICD-9 catalog. 

In [5]:
meta_dir = wd / "data" / "meta"
meta_dir.mkdir(exist_ok=True, parents=True)

meta_url = "https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Publications/ICD9-CM/2011/Dtab12.zip"
with ZipFile(BytesIO(r.get(meta_url, stream=True).content)) as zip_ref:
    zip_ref.extractall(meta_dir)

Read in the rtf format of the catalog. This takes several minutes!

In [6]:
from striprtf.striprtf import rtf_to_text

icd_file = meta_dir / "dtab12.rtf"

with open(icd_file, "r") as f:
    icd_catalog = rtf_to_text(f.read())

Parse the catalog into a hierarchical structure that suits conquery as a concept. We define the `concept` foundation and then add the children tree to it.
Because the catalog has already the sorted structure we define state variables (`in_chapter`, `in_section`, ...) to keep track of where we have been recently when we insert new nodes to the tree.

An ICD-9 code can have up to 5 levels. The first two levels are specified by a range in which a fixed length prefix might fall. All lower sections are determined by a single fixed length prefix. Fortunately, we can distinguish lines in the catalog regarding their level by matching each line to a distinct RegEx schema (`chapter_matcher`, `section_matcher`, ...).
The lines that do not match are treated as addition infos and are appended to the last parsed element `in_recent`.
If present, we add additional infos to each node, which is displayed in the left column in the frontend. For the ICD-9 codes, this usually includes indication whether the code can be applied using *Exclude* and *Include* sections.

In [8]:
chapter_matcher = re.compile(r"^(?P<chapter>\d+\.)\s*(?P<name>[A-Z, -]+)\((?P<start>\d{3})-(?P<end>\d{3})\)$")
section_matcher = re.compile(r"^(?P<name>[A-Z, -]+)\((?P<start>\d{3})-(?P<end>\d{3})\)$")
subsection_matcher = re.compile(r"^(?P<prefix>[\d]{3})\s+(?P<name>[\w\d ()\-,\[\]\.]+)")
subsubsection_matcher = re.compile(r"^(?P<prefix>[\d]{3}\.\d)\s+(?P<name>[\w\d ()\-,\[\]\.]+)")
subsubsubsection_matcher = re.compile(r"^(?P<prefix>[\d]{3}\.\d{2})\s+(?P<name>[\w\d ()\-,\[\]\.]+)")

# The concepts builds the root of the hierarchy
concept = {
    # Placeholder for the rest of the tree
    "children": [],
    # Defines which columns represent codes for this concept
    "connectors": [{
        "column": "DIAGNOSES_ICD.icd9_code",
        "label": "Diagnoses"
    }],
    # The display name
    "label" : "ICD",
    # The internal name that is used to create an id (must be unique in a dataset among concepts, tables and secondaryIds)
    "name" : "icd",
    # Selects define aggregations that create additional columns in the output
	"selects": [],
    # At the moment there is just this type TREE
	"type": "TREE"
}

in_chapter = None
in_section = None
in_subsection = None
in_subsubsection = None
in_subsubsubsection = None
in_recent = None
additional_info_key = ""
for line in icd_catalog.split("\n") :

    # Chapter
    match = chapter_matcher.match(line)
    if match:
        in_chapter = {
            # The label will be displayed in the concept overview, the query editor and the query result
            "label": f"{match.group('chapter')}",
            # The description is displayed in the concept overview, the query editor
            "description": match.group("name").title(),
            "condition": {
                # Defines with codes fall into this chapter/section 
                "type": "PREFIX_RANGE",
                "min": match.group("start"),
                "max": match.group("end"),
            },
            # Placeholer for sections, the next level in the tree
            "children": [],
            # Placeholder for 
            "additionalInfos": [],
        }
        in_recent = in_chapter
        # Reset sub-levels
        in_section = None
        in_subsection = None
        in_subsubsection = None
        in_subsubsubsection = None
        additional_info_key = ""
        concept["children"].append(in_chapter)
        continue

    # Section
    match = section_matcher.match(line)
    if match:
        in_section = {
            "label": f"{match.group('start')}-{match.group('end')}",
            "description": match.group("name").title(),
            "condition": {
                "type": "PREFIX_RANGE",
                "min": match.group("start"),
                "max": match.group("end"),
            },
            "children": [],
            "additionalInfos": [],
        }
        in_recent = in_section
        in_subsection = None
        in_subsubsection = None
        in_subsubsubsection = None
        additional_info_key = ""
        in_chapter["children"].append(in_section)
        continue

    # Subsection
    match = subsection_matcher.match(line)
    if match:
        in_subsection = {
            "label": match.group("prefix"),
            "description": match.group("name"),
            "condition": {
                "type": "PREFIX_LIST",
                "prefixes": [match.group("prefix")],
            },
            "children": [],
            "additionalInfos": [],
        }
        in_recent = in_subsection
        in_subsubsection = None
        in_subsubsubsection = None
        additional_info_key = ""

        upper_level = in_section or in_chapter
        upper_level["children"].append(in_subsection)
        continue

    # Subsubsection
    match = subsubsection_matcher.match(line)
    if match:
        in_subsubsection = {
            "label": match.group("prefix"),
            "description": match.group("name"),
            "condition": {
                "type": "PREFIX_LIST",
                # the descriptive codes differ from the codes in the data in that they contain a dot that we strip
                "prefixes": [match.group("prefix").replace(".","")],
            },
            "children": [],
            "additionalInfos": [],
        }
        in_recent = in_subsubsection
        in_subsubsubsection = None
        additional_info_key = ""

        upper_level = in_subsection or in_section or in_chapter
        upper_level["children"].append(in_subsubsection)
        continue

    # Subsubsubsection
    match = subsubsubsection_matcher.match(line)
    if match:
        in_subsubsubsection = {
            "label": match.group("prefix"),
            "description": match.group("name"),
            "condition": {
                "type": "PREFIX_LIST",
                "prefixes": [match.group("prefix").replace(".","")],
            },
            "children": [],
            "additionalInfos": [],
        }
        in_recent = in_subsubsubsection
        additional_info_key = ""

        
        upper_level = in_subsubsection or in_subsection or in_section or in_chapter
        upper_level["children"].append(in_subsubsubsection)
        continue

    # Additional Infos
    if not in_recent:
        continue
    value = line
    if line.startswith('Excludes:'):
        additional_info_key = "Excludes:"
        # +1 for the \t 
        value = value[len(additional_info_key)+1:]
    elif line.startswith('Includes:'):
        additional_info_key = "Includes:"
        value = value[len(additional_info_key)+1:]

    additional_info = in_recent["additionalInfos"]

    # Should be one item at max
    items = list(filter(lambda i: i["key"] == additional_info_key, additional_info))

    if len(items) > 1:
        raise RuntimeError(f"Expected key {additional_info_key} to appear at most once")
    if len(items) == 0:
        # First time this key appeared
        additional_info.append(
            {
                "key": additional_info_key,
                "value": value
            }
        )
        continue

    items[0]["value"] += f"\n{value}"
    

Validate and write the concept. //TODO

In [10]:

concept_json_file = wd / "data" / "concepts" / "icd.concept.json"
concept_json_file.parent.mkdir(parents=True, exist_ok=True)

with open(concept_json_file, "w") as f:
    json.dump(concept, f, indent="\t")

## Preprocessing

This step assumes that the backend has already been build/compiled (using the script `conquery/scripts/build_no_version.sh`).

The command that is executed converts the CSV we downloaded initially to an CQPP.

In [ ]:
(wd / "data" / "cqpp").mkdir(exist_ok=True, parents=True)
!java -jar ../../executable/target/executable-0.0.0-SNAPSHOT.jar preprocess --desc ./data/import* --in . --out ./data/cqpp

## Import/Upload

This step assumes, that the backend is running and the admin endpoint is reachable under the specified url (e.g. using the script `conquery/scripts/run_conquery_cypress.sh`).

### Meta Data Import

In [ ]:
import time

# Create Dataset
datasetId = "mimic-iii-demo"
r.post("http://localhost:8081/admin/datasets", json={"name":"mimic-iii-demo", "label": "MIMIC-III Demo"}, headers={"content-type":"application/json"})
time.sleep(2)
r.post(f"http://localhost:8081/admin/datasets/{datasetId}/tables", json=table, headers={"content-type":"application/json"})
time.sleep(2)
r.post(f"http://localhost:8081/admin/datasets/{datasetId}/concepts", json=concept, headers={"content-type":"application/json"})


### Data Import

In [ ]:
with open(wd / "data" / "cqpp" / "DIAGNOSES_ICD.cqpp", "rb") as f :
    r.post(f"http://localhost:8081/admin/datasets/{datasetId}/cqpp", data= f, headers={"content-type":"application/octet-stream"})

### Update Matching Stats

This action collects statistics that are displayed in the frontend when hovering over concepts. 

In [ ]:

r.post(f"http://localhost:8081/admin/datasets/{datasetId}/update-matching-stats", headers={"content-type":"application/json"})

## Visit the Frontend

Finally you can access the frontend under the url http://localhost:8000/?access_token=user.SUPERUSER@SUPERUSER as the super user. In the top right corner choose the *MIMIC-III Demo* dataset. You can then start combining nodes of the *ICD* concept in the query editor and submit your query.